In [8]:
# Loading model
from langchain_groq import ChatGroq
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
load_dotenv()

True

In [152]:
llm = ChatGroq(
    model_name ="openai/gpt-oss-20b",
    temperature = 0
)
# llm = ChatOllama(model="qwen:7b",temperature = 0)

In [153]:
response = llm.invoke("capital of Londan?")

In [154]:
response.content

'London is a city, not a country, so it doesn’t have a “capital.”  \n- **London** is the capital city of **England** (and of the United Kingdom as a whole).'

In [155]:
from langchain.tools import tool

In [156]:
@tool
def multiply(a:int,b:int) ->int:
    """multiply two integer
    Args:
        a (int):The first integer
        b (int):The second integer

    return:
        int:The product of a and b
    """
    return a*b


In [157]:

@tool
def add(a:int,b:int) ->int:
    """addition two integer
    Args:
        a (int):The first integer
        b (int):The second integer

    return:
        int:The addition of a and b
    """
    return a+b


In [158]:


@tool
def divide(a:int,b:int) ->float:
    """divition two integer
    Args:
        a (int):The first integer
        b (int):The second integer

    return:
        int:The divide of a and b
    """
    if b ==0:
        raise ValueError("Denominator cannot be zero")
        
    return a/b




# Importing the inbuilt tool

In [159]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

In [160]:
search.invoke("What is latest update on iphone7?")

'Latest Singapore news, including the city state’s relationships with Malaysia and Mahathir, China and Xi Jinping, and the rest of Southeast Asia. Sep 26, 2025 · The latest military news, insights and analysis from China. Latest China military news, including US-China military tensions, China’s spending on defence and Beijing’s vow to increase its preparation for war. Latest news, in-depth features and opinion on Malaysia, covering politics, economy, society and the Asean member-nation’s relationships with China, Singapore, and other Southeast Asian … All the latest news, opinions and analysis on Hong Kong, China, Asia and around the world'

In [161]:
tools = [search, multiply, add, divide]

In [162]:
llm_with_tool = llm.bind_tools(tools)

In [163]:
response = llm_with_tool.invoke("what is a current age of the TATA Group?")

In [164]:
response.content

'The Tata Group was founded in 1868 by Jamsetji\u202fTata.  \nAs of 2025, that makes the group **157\u202fyears old** (2025\u202f–\u202f1868\u202f=\u202f157).'

In [166]:
response.tool_calls

[]

In [167]:
from langgraph.graph import StateGraph,END,MessagesState,START
from langchain_core.messages import HumanMessage,SystemMessage

In [168]:
# Create system prompt
SYSTEM_PROMPT = "You are a helpful assistant tasked with using search and performing arithmetic on a set of inputs."

In [181]:
def function_1(state:MessagesState):
    user_query = state["messages"]
    input = [SYSTEM_PROMPT]+user_query
    response=llm_with_tool.invoke(input)
    return {"messages":[response]}

In [182]:
builder = StateGraph(MessagesState)

In [183]:
# llm is used as validator
builder.add_node("LLM_Decision_Maker",function_1)

In [184]:
from langgraph.prebuilt import ToolNode
builder.add_node("tools",ToolNode(tools))

In [185]:
builder.add_edge(START,"LLM_Decision_Maker")

In [186]:
from langgraph.prebuilt import tools_condition
builder.add_conditional_edges("LLM_Decision_Maker",tools_condition)

In [187]:
builder.add_edge("tools","LLM_Decision_Maker")

In [188]:
app =builder.compile()

In [189]:
message=[HumanMessage(content="what is 2+2??")]

# passing message
app.invoke({"messages":message})

{'messages': [HumanMessage(content='what is 2+2??', additional_kwargs={}, response_metadata={}, id='e42f8f51-54da-413c-8e52-a060972013d6'),
  AIMessage(content='', additional_kwargs={'reasoning_content': "We need to answer 2+2. It's simple arithmetic. Use add function.", 'tool_calls': [{'id': 'fc_2bfd458b-d3a4-4de6-8003-289c7cde8392', 'function': {'arguments': '{"a":2,"b":2}', 'name': 'add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 352, 'total_tokens': 396, 'completion_time': 0.045349016, 'prompt_time': 0.005856055, 'queue_time': 0.043250521, 'total_time': 0.051205071, 'prompt_tokens_details': {'cached_tokens': 256}}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--9e8b8973-14c5-429f-b40b-31bda9baf675-0', tool_calls=[{'name': 'add', 'args': {'a': 2, 'b': 2}, 'id': 'fc_2bfd458b-d3a4-4de6-8003-289c7cde8392', 'type':

In [194]:
message = [HumanMessage(content="How much is the net worth of Elon Musk, and divide it by 2?")]

# passing message
messages = app.invoke({"messages": message})

for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

How much is the net worth of Elon Musk, and divide it by 2?
================================== Ai Message ==================================
Tool Calls:
  duckduckgo_search (fc_d43f46da-408c-40f3-919b-0000b93b4cba)
 Call ID: fc_d43f46da-408c-40f3-919b-0000b93b4cba
  Args:
    query: Elon Musk net worth
================================= Tool Message =================================
Name: duckduckgo_search

1 week ago - He was first listed on the Forbes Billionaires List in 2012, with a net worth of $2 billion . Musk paid $455 million in taxes on $1.52 billion of income between 2014 and 2018. According to ProPublica, Musk paid no federal income taxes in 2018. 2 days ago - Elon Reeve Musk (/ˈiːlɒn/ EE-lon; born June 28, 1971) is a businessman and entrepreneur known for his leadership of Tesla, SpaceX, Twitter, and xAI. Musk has been the wealthiest person in the world since 2021; as of October 2025, Forbes e

In [191]:
messages = [HumanMessage(content="what is 2 time of age of narendra modi ji")]
messages = app.invoke({"messages": messages})

In [192]:
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

what is 2 time of age of narendra modi ji
================================== Ai Message ==================================
Tool Calls:
  duckduckgo_search (fc_e3c1b2e2-ae8e-4a89-acfc-d9e8adc15ac0)
 Call ID: fc_e3c1b2e2-ae8e-4a89-acfc-d9e8adc15ac0
  Args:
    query: Narendra Modi age 2025
================================= Tool Message =================================
Name: duckduckgo_search

Prime Minister Narendra Modi turned 75 on September 17, 2025 . Know his exact age , date of birth, records as India's longest-serving non-Congress PM, and details on the PM age limit in India., India News, Times Now PM Modi turns 75 and proves that age is just a number As Prime Minister Narendra Modi turns 75, his whirlwind schedule across five states proves that age is truly just a number. From addressing rallies to strategising with military leaders, his energy remains undiminished Lt Gen (Retd) MK Das Sep 17, 2025 

In [195]:
messages = [HumanMessage(content="can you give me 2 times of current stock price of Apple with the latest news of the Apple.")]
messages = app.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

can you give me 2 times of current stock price of Apple with the latest news of the Apple.
================================== Ai Message ==================================
Tool Calls:
  duckduckgo_search (fc_5e337f0e-08ca-46f1-858b-e5d524e16209)
 Call ID: fc_5e337f0e-08ca-46f1-858b-e5d524e16209
  Args:
    query: Apple stock price current
================================= Tool Message =================================
Name: duckduckgo_search

Discover the innovative world of Apple and shop everything iPhone, iPad, Apple Watch, Mac, and Apple TV, plus explore accessories, entertainment and expert device support. Celebrate with up to ₹10000 instant cashback with eligible cards. Plus up to 12 months of No Cost EMI. Shop now at apple.com. The latest Apple Watch models can show you its approximate distance and direction. 13 To set up a group photo on your iPhone, join the group and use Apple Watch as a viewfin